In [1]:
import os, shutil
import numpy as np
from angstrom import Molecule, Trajectory
from copy import deepcopy
from surface_scan import read_datafile, get_lammps_data_lines, change_coordinates, write_job_file, write_spring_input

### Select scanning location

In [ ]:
start = [21.582, 22.8912, 13]
ydist = (25.4346 - 22.8912)
# ydist = 0
# xdist = (25.179 - 21.582)
xdist = 0
dx, dy = 0.5, 0.1

In [ ]:
xpos = np.arange(start[0], start[0] + xdist + dx, dx)
ypos = np.arange(start[1], start[1] + ydist + dy, dy)
print('x: %i points | y: %i points | Total: %i points' % (len(xpos), len(ypos), len(xpos) * len(ypos)))

### Read data file
Atomic coordinates will be used to move the molecule along the surface

In [ ]:
scandir = '/home/kutay/Documents/git/Nanocar/scripts/surface-scan/lammps'
srcdir = '/home/kutay/Documents/git/Nanocar/scripts/surface-scan/HtBDC_Cu110'
datafile = os.path.join(srcdir, 'data.HtBDC_Cu110')
# infile = os.path.join(srcdir, 'in.HtBDC_Cu110_min')
# infile = os.path.join(srcdir, 'in.HtBDC_Cu110_spring')
infile = os.path.join(srcdir, 'in.HtBDC_Cu110_rigid_spring')
jobfile = os.path.join(srcdir, 'job.HtBDC_Cu110')
elements = {'1': 'Cu', '2': 'C', '3': 'H', '4': 'C'}
atomids, coords = read_datafile(datafile)
atoms = [elements[i] for i in atomids]

### Create Lammps files

In [ ]:
# Center molecule to starting position to use as reference
mol = Molecule(atoms=atoms, coordinates=coords)
mol.center(start)

nframes = int(len(xpos) * len(ypos))
natoms = len(coords)
# Write trajectory to make sure everything is right
traj = Trajectory()
traj.coordinates = np.empty((nframes, natoms, 3))
traj.atoms = np.empty((nframes, natoms), dtype='U2')
# Spring constants (x and y is changes for each simulation, z is selected from eq position)
SPRING = {'k': 10.0, 'xeq': 0.0, 'yeq': 0.0, 'zeq': 'NULL', 'r0': 0.0}

In [ ]:
idx = 0
for yi, y in enumerate(ypos):
    for xi, x in enumerate(xpos):
        print('\r%i / %i | x: %.2f | y: %.2f' % (idx + 1, nframes, x, y), end='')
        # initialize new directory
        simdir = os.path.join(scandir, '%i-%i-%i' % (idx, xi, yi))
        os.makedirs(simdir, exist_ok=True)
        # position the molecule
        newmol = Molecule(atoms=deepcopy(mol.atoms), coordinates=deepcopy(mol.coordinates))
        newmol.center([x, y, start[2]])
        # create new data file
        newdatafile = os.path.join(simdir, 'data.HtBDC_Cu110')
        newlines = get_lammps_data_lines(newmol.coordinates, atomids, startid=1401)
        change_coordinates(datafile, newdatafile, newlines, start=1464)
        # add input file
        newinfile = os.path.join(simdir, 'in.HtBDC_Cu110')
        # shutil.copy(infile, newinfile)
        # Change spring coefficient (use only with spring)
        SPRING['xeq'], SPRING['yeq'] = x, y
        write_spring_input(infile, newinfile, SPRING, springindex=48)
        # add job file
        newjobfile = os.path.join(simdir, 'job.HtBDC_Cu110')
        write_job_file(jobfile, newjobfile, '%i-%i-%i' % (idx, xi, yi))
        # Save frame to trajectory
        traj.coordinates[idx] = newmol.coordinates
        traj.atoms[idx] = newmol.atoms
        idx += 1

In [ ]:
traj.write('traj.xyz')

### Plot points

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
centers = traj.get_center()

In [ ]:
xpos = [i[0] for i in centers]
ypos = [i[1] for i in centers]

In [ ]:
surfatoms1 = [[21.582, 22.8912], [25.179, 22.8912], [21.582, 25.4346], [25.179, 25.4346]]
surfatoms2 = [23.3805, 24.1629]

In [ ]:
plt.scatter(xpos, ypos, alpha=0.9)
plt.scatter([i[0] for i in surfatoms1], [i[1] for i in surfatoms1], c='r', alpha=0.9, s=200)
plt.scatter(surfatoms2[0], surfatoms2[1], c='r', alpha=0.5, s=200)